# Common (Polyglot)

In [ ]:
#!import ../../lib/fsharp/Notebooks.dib
#!import ../../lib/fsharp/Testing.dib

In [ ]:
#if !INTERACTIVE
open Lib
#endif

In [ ]:
let nl = System.Environment.NewLine
let q = @""""

In [ ]:
let inline cons head tail = head :: tail

## memoize

In [ ]:
let inline memoize fn =
    let result = lazy fn ()
    fun () -> result.Value

## TraceLevel

In [ ]:
type TraceLevel =
    | Verbose
    | Debug
    | Info
    | Warning
    | Critical

let inline _locals () = ""

## trace

In [ ]:
let to_trace_level = function
    | Verbose -> SpiralTrace.TraceLevel.US0_0
    | Debug -> SpiralTrace.TraceLevel.US0_1
    | Info -> SpiralTrace.TraceLevel.US0_2
    | Warning -> SpiralTrace.TraceLevel.US0_3
    | Critical -> SpiralTrace.TraceLevel.US0_4

let trace level fn locals =
    let level = level |> to_trace_level
    SpiralTrace.trace level fn locals

In [ ]:
//// test

trace Debug (fun () -> "test") _locals

00:00:00 d #1 test
